In [90]:
import pandas as pd
from sklearn import datasets, svm, metrics
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [91]:
avila =  pd.read_csv('./avila-bible-datamad1019/training_dataset.csv')

In [92]:
avila.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,scribe
0,0,-0.091897,0.297600,0.079145,0.196496,0.261718,1.269960,0.446679,-0.751707,0.001721,0.998901,Philippus
1,1,-0.091897,0.226939,0.267634,0.024091,0.261718,-0.806282,0.597681,-0.601277,0.126447,-0.909619,Paithonius
2,2,0.167323,0.313302,0.168055,-0.383198,0.261718,0.190314,0.824183,0.558250,-0.247731,-0.148073,Marcus
3,3,-0.017834,-0.228430,0.370770,1.293671,0.172340,0.896237,0.182426,0.416867,1.373706,0.868284,Noaelius
4,4,0.043885,0.407516,-0.120014,0.281743,0.261718,-0.183409,0.106925,0.142896,0.531806,-0.101311,Marcus


In [93]:
scribes = {
    "Marcus": 1, 
    "Philippus": 2,
    "Franciscus": 3, 
    "Begonius": 4, 
    "Noaelius": 5,
    "Paithonius": 6, 
    "Ubuntius": 7, 
    "FerrumEffractarius": 8
}

In [96]:
for i in range(len(avila)):
    n = scribes[avila.loc[i, "scribe"]]
    avila.loc[i, "scribe_num"] = n

In [109]:
X = avila[["F1","F2","F3","F4","F8","F9","F10"]]

In [110]:
X

,F1,F2,F3,F4,F8,F9,F10
0,-0.091897,0.297600,0.079145,0.196496,-0.751707,0.001721,0.998901
1,-0.091897,0.226939,0.267634,0.024091,-0.601277,0.126447,-0.909619
2,0.167323,0.313302,0.168055,-0.383198,0.558250,-0.247731,-0.148073
3,-0.017834,-0.228430,0.370770,1.293671,0.416867,1.373706,0.868284
4,0.043885,0.407516,-0.120014,0.281743,0.142896,0.531806,-0.101311
...,...,...,...,...,...,...,...
12012,-0.165960,0.007106,-2.172058,-0.165858,1.695605,0.812439,-0.471271
12013,0.463576,0.109171,0.224957,-1.411178,0.925104,0.656532,-0.493753
12014,3.265629,-0.251983,0.125378,-2.123860,-0.464845,-0.123005,0.721958
12015,0.142636,-0.330495,0.299642,0.418282,-0.021164,-0.996086,-1.545732


In [117]:
y = avila.scribe

In [118]:
y

0         Philippus
1        Paithonius
2            Marcus
3          Noaelius
4            Marcus
            ...    
12012    Franciscus
12013        Marcus
12014      Begonius
12015        Marcus
12016        Marcus
Name: scribe, Length: 12017, dtype: object

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [120]:
y_test.isna().sum()

0

In [121]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [122]:
models = {
    "svm": LinearSVC(),
    "logistic": LogisticRegression(solver='lbfgs', max_iter=2000),
    "forest": RandomForestClassifier()
}

for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)

Training model: svm


/home/mariobru/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mariobru/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training model: logistic
Training model: forest


/home/mariobru/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [123]:
d = {modelName:model.predict(X_test) for modelName, model in models.items()}

df = pd.DataFrame(d)
df["gt"] = y_test
df

,svm,logistic,forest,gt
0,Marcus,Marcus,Philippus,Philippus
1,Marcus,Marcus,Philippus,Paithonius
2,Marcus,Marcus,Marcus,NaN
3,Marcus,Marcus,Paithonius,NaN
4,Marcus,Begonius,Begonius,NaN
...,...,...,...,...
2399,Marcus,Marcus,Marcus,Marcus
2400,Marcus,Marcus,Marcus,NaN
2401,Marcus,Marcus,Philippus,NaN
2402,Marcus,Marcus,Marcus,NaN


In [116]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from IPython.core.display import display, HTML

switchColor = lambda value: f"<b style=\"color:orange\">{round(value,2)}</b>" if value < 0.8 else f"<b style=\"color:green\">{round(value,2)}</b>"
printBonito = lambda label,val: display(HTML(f"<span style=\"padding-left:20px\">The {label} is: {switchColor(val)}</span>"))

for modelName, model in models.items():    
    print(f"Evaluating model [{modelName}]:")
    printBonito("Accuracy", accuracy_score(df["gt"],df[modelName]))
    printBonito("Precision", precision_score(df["gt"],df[modelName],average='weighted'))
    printBonito("Recall", recall_score(df["gt"],df[modelName],average='weighted'))

Evaluating model [svm]:


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets